<a href="https://colab.research.google.com/github/abhishekvarma33/Deep-Learning---Additional-use-cases-/blob/master/Med_Analytics_EQUINIX_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This Notebook includes
1) Packages Installation for R
2) Load Libraries
3) Data Preparation and EDA
4) Data Preprocessing
5) Time series decomposition for model training
6) Time series Forecasting
7) Evaluation and model improvement
8) Final Prediction, Insights and Results

#1) Package Installation

In [1]:
install.packages('IRkernel')
IRkernel::installspec(user = FALSE)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
install.packages(c('forecast', 'tseries'))

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘xts’, ‘TTR’, ‘fracdiff’, ‘lmtest’, ‘Rcpp’, ‘timeDate’, ‘urca’, ‘zoo’, ‘RcppArmadillo’, ‘quadprog’, ‘quantmod’




In [ ]:

# Load necessary libraries
library(forecast)
library(tseries)
library(tidyverse)
library(lubridate)
library(readr)


Upload CSV files and load into dataframe

In [ ]:
# Specify the path to the uploaded CSV file
csv_path <- "/content/bill_amount.csv"  # Update with the actual path

# Read the CSV file into a data frame
data_bill_amount <- read_csv(csv_path)

In [ ]:
csv_path <- "/content/bill_id.csv"  # Update with the actual path

# Read the CSV file into a data frame
data_bill_id <- read_csv(csv_path)

In [ ]:
csv_path <- "/content/clinical_data.csv"  # Update with the actual path

# Read the CSV file into a data frame
data_clinical <- read_csv(csv_path)

In [ ]:
csv_path <- "/content/demographics.csv"  # Update with the actual path

# Read the CSV file into a data frame
data_demographics <- read_csv(csv_path)

In [ ]:
summary(data_bill_id)

In [ ]:
summary(data_bill_amount)

In [ ]:
summary(data_clinical)

In [ ]:
summary(data_demographics)

Vaidating Duplicates

In [ ]:
# Check for duplicates in the entire data_bill_amount data frame
duplicates <- duplicated(data_bill_amount$bill_id)

# Display the rows with duplicates
data_bill_amount[duplicates, ]

In [ ]:
# Check for duplicates in the entire data_bill_amount data frame
duplicates <- duplicated(data_bill_id$bill_id)

# Display the rows with duplicates
data_bill_id[duplicates, ]

Joining all CSV files to achieve a single data set

In [ ]:

# Perform an outer join to include all data fields
df_bill <- merge(data_bill_id, data_bill_amount, by = "bill_id", all = TRUE)

# Display the merged data frame
summary(df_bill)


Joined bill and bill_amounts using outer join [using bill_id as primary key]
In next step joining demographics and joined bill df [using patient id as primary key]

In [ ]:
# Perform an outer join to include all data fields
df_bill_demo <- merge(df_bill, data_demographics, by = "patient_id", all = TRUE)

# Display the merged data frame
summary(df_bill_demo)

In [ ]:
data_clinical <- data_clinical %>%
  rename(patient_id = id)

In [ ]:
# Perform an outer join to include all data fields
df <- merge(df_bill_demo, data_clinical, by = "patient_id", all = TRUE)

# Display the merged data frame
summary(df)

In [ ]:
#Joined all dataframes into df and ready top use a combined dataset and perform data preparation and preprocessing

In [ ]:
# Convert "yes" to 1 and "no" to 0
df$medical_history_3 <- ifelse(tolower(df$medical_history_3) == "yes", 1, ifelse(tolower(df$medical_history_3) == "no", 0, df$medical_history_3))

# Print the modified dataframe
summary(df)

In [ ]:
# Convert "Male" to m and "Female" to f
df$gender <- ifelse(tolower(df$gender) == "male", 'm', ifelse(tolower(df$gender) == "female", 'f', df$gender))


In [ ]:
# Convert "India" to Indian
df$race <- ifelse(tolower(df$race) == "india", 'Indian', df$race)

# Convert "Singapore citizen" to Singaporean
df$resident_status <- ifelse(tolower(df$resident_status) == "singapore citizen", 'Singaporean', df$resident_status)

In [ ]:
# Calculate the difference in days between admissing and discharge date to achieve no of days a patient spent in hospital
df$days_difference <- as.numeric(difftime(df$date_of_discharge,df$date_of_admission.y, units = "days"))
summary(df$days_difference)

In [ ]:

df$date_of_admission.y <- as.Date(df$date_of_admission.y)
df$date_of_discharge <- as.Date(df$date_of_discharge)
# Convert the date format to 'yyyy-mm-dd' using as.Date
df$dob  <- as.Date(df$date_of_birth , format = '%d-%m-%Y')

# Print the converted date
year(df$dob)


In [ ]:
# Calculate age using date of birth and date of admission
# Load the lubridate package

df$year_birth=year(df$dob)
df$year_adm= year(df$date_of_admission.y)
df$age <- as.numeric(df$year_adm-df$year_birth)

In [ ]:
# Group by patient_id and calculate sum(amount) and retain other fields
result <- df %>%
  group_by(patient_id) %>%
  summarise(total_amount = sum(amount))

# Print the result
print(result)

In [ ]:
# Merge the data frames based on patient_id
merged_df <- result %>%
  inner_join(df, by = "patient_id")

In [ ]:
head(merged_df,2)

In [ ]:
head(df,5); head(merged_df,5)


In [ ]:
# Calculate sum of amounts grouped by year
sum_by_month <- df %>%
  group_by(year = lubridate::month(date_of_admission.y)) %>%
  summarise(total_amount_monthly = sum(amount))
print(sum_by_month)

In [ ]:
# Calculate sum of amounts grouped by year
sum_by_year <- df %>%
  group_by(year = year_adm) %>%
  summarise(total_amount = sum(amount))
print(sum_by_year)

In [ ]:
summary(merged_df)

#OUTLIER

In [ ]:
filtered_data <- merged_df %>%
  filter(amount > 122200)
filtered_data

In [ ]:
# Change the amount for patient iD which is outlier above
merged_df <- merged_df %>%
  mutate(amount = ifelse(patient_id == '86e3adc71c41b050f83fbe3d34d949e7', 10075, amount))

# View the modified dataframe
summary(merged_df)



In [ ]:
library(tidyverse)
merged_df %>%
  ggplot(aes(x = date_of_discharge, y = amount)) +
  geom_line() +
  labs(title = "Sales Over Time")


In [ ]:
merged_df %>%
  ggplot(aes(x = date_of_discharge, y = amount, color = gender)) +
  geom_line() +
  labs(title = "Sales Over Time by Gender")
#scale_x_date(date_breaks = "1 month", date_labels = "%b %Y")  # Set monthly scale

In [ ]:
merged_df %>%
  ggplot(aes(x = date_of_discharge, y = amount)) +
  geom_line() +
  facet_wrap(~ gender, scales = "free_y") +
  labs(title = "Sales Over Time by Gender (Faceted)")

In [ ]:
merged_df %>%
  ggplot(aes(x = race, y = amount, fill = race)) +
  geom_boxplot() +
  labs(title = "Sales Distribution by Race")

In [ ]:
install.packages("forecast")
install.packages("ggplot2")
library(forecast)
library(ggplot2)

In [ ]:
# Create a time series object
sales_ts <- ts(merged_df$amount, frequency = 365)  # Assuming daily frequency

In [ ]:
sales_ts

In [ ]:


library(lubridate)
library(forecast)
hospital_data=df
# Convert 'date' to a proper date format
hospital_data$date <- as.Date(hospital_data$date_of_discharge)

# Group data by month and calculate total revenue
monthly_revenue <- hospital_data %>%
  group_by(month = floor_date(date, unit = "month")) %>%
    summarise(total_revenue = sum(amount))


In [ ]:
monthly_revenue

In [ ]:
# Assuming your data frame is named 'your_data_frame'
cleaned_data <- na.omit(monthly_revenue)

In [ ]:
#Convert to a time series object
ts_data <- ts(cleaned_data$total_revenue, frequency = 12, start = c(year(min(cleaned_data$month)), month(min(cleaned_data$month))))

In [ ]:
print(ts_data)

In [ ]:
c(year(max(cleaned_data$month))-1 , month(max(cleaned_data$month))+10)

In [ ]:
c(year(min(cleaned_data$month)), month(max(cleaned_data$month)))

In [ ]:
# Split data into training and test sets
train_size <- length(ts_data)  # Assuming you're forecasting for one year
train <- window(ts_data, end = c(year(max(cleaned_data$month)), month(max(cleaned_data$month))))
test <- window(ts_data, start = c(year(min(cleaned_data$month)), month(max(cleaned_data$month))))


In [ ]:
# Fit an auto ARIMA model
fit <- auto.arima(train)


In [ ]:
# Forecasting for 12 months
forecast <- forecast(fit, h = 12)


In [ ]:
# Calculate performance metrics
accuracy <- accuracy(forecast, test)
print(accuracy)


In [ ]:
forecast

In [ ]:
         Point Forecast   Lo 80   Hi 80    Lo 95   Hi 95
Dec 2015        1526671 1113434 1939907 894680.1 2158661
Jan 2016        1526671 1113434 1939907 894680.1 2158661
Feb 2016        1526671 1113434 1939907 894680.1 2158661
Mar 2016        1526671 1113434 1939907 894680.1 2158661
Apr 2016        1526671 1113434 1939907 894680.1 2158661
May 2016        1526671 1113434 1939907 894680.1 2158661
Jun 2016        1526671 1113434 1939907 894680.1 2158661
Jul 2016        1526671 1113434 1939907 894680.1 2158661
Aug 2016        1526671 1113434 1939907 894680.1 2158661
Sep 2016        1526671 1113434 1939907 894680.1 2158661
Oct 2016        1526671 1113434 1939907 894680.1 2158661
Nov 2016        1526671 1113434 1939907 894680.1 2158661